In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import v2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
batch_size = 32

In [4]:
# Simple data augmentation and transformation for now 

transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True)
])

In [5]:
# 60k train_dataset, 10k test_dataset from MNIST

train_dataset = torchvision.datasets.MNIST(root='../dataset/', train=True, transform=transforms, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='../dataset/', train=False, transform=transforms, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)   

In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
# Hyperparameters
input_size = 784 # 28x28 pixels for each MNIST image
num_classes = 10 # prediction 0-9 digits
learning_rate = 0.001
num_epochs = 5

# Initialise Network
model = MLP(input_size=input_size, num_classes=num_classes).to(device)

# Loss and Optimiser
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# Training the model

for epoch in range(num_epochs):
    print(f'Epoch: {epoch}')
    for batch_indx, (data, targets) in enumerate(train_loader): # (data, targets) == (input features, targets)
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # print(data.shape)  # => [64 , 1, 28, 28] => 64 : num_images aka batch_size, 1 -> num_channels, (28,28): (height, width)
        data = data.reshape(data.shape[0], -1) # Flatten from [batch_size, 1, 28, 28] to [batch_size, 784]
        if epoch == 0 and batch_indx == 0: 
            print(data.shape)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward 
        optimiser.zero_grad() # before the backward pass, all gradients of the model's parameters are reset to zero. This is necesasry because by defaut, gradients are accumulated by PyTorch.
        loss.backward()

        # gradient desecent
        optimiser.step()


Epoch: 0
torch.Size([32, 784])
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4


In [12]:
def check_accuracy(loader, model): 
    if loader.dataset.train: 
        print("Accuracy on training data")
    else:
        print("Accuracy on testing data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader: 
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0],-1)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100: .2f}')
    
    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


Accuracy on training data
Got 58785/60000 with accuracy  97.97
Accuracy on testing data
Got 9704/10000 with accuracy  97.04
